# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [ ]:
# Изучим тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
# Изменю тип данных столбцов 'calls' и 'messages' на int
df['calls']=df['calls'].astype("int")
df['messages']=df['messages'].astype("int")

In [ ]:
# Проверим
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [ ]:
# Проверим на дубликаты
df.duplicated().sum()

0

**Вывод:**

В имеющихся данных отсутствуют пропуски и дубликаты, предобработки не требуется.

Единственное изменил тип данных в столбцах 'calls' и 'messages', т.к. логично что количество звонков и сообщений должно быть выражено в целых числах

## Разбейте данные на выборки

Разбиваю имеющуюся базу днных на выборки в соотношении 3(train):1(val):1(test)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)
features = df_test.drop('is_ultra', axis=1)
target = df_test['is_ultra']
features_valid, features_test, target_valid, target_test = train_test_split(features, target,
                                                                            test_size=0.5, random_state=12345)

In [ ]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

Описание данных:

features_train, target_train - обучающая выборка

features_valid, target_valid - валидационная выборка

features_test, target_test - тестовая выборка

## Исследуйте модели

**Логистическая регрессия**

In [ ]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)

Качество: 0.7107309486780715


**Случайный лес**

In [ ]:

best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,51,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est,
                                           min_samples_leaf=sample, random_state=12345)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:",
      best_est, "   мин. объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 6     Количество деревьев: 35    мин. объектов в узле: 2   Качество: 0.8087091757387247


**Дерево решений**

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Качество:", best_accuracy)

Глубина дерева: 3     Качество: 0.7853810264385692


**Вывод:**

Лучшей моделью по метрике accuracy (Качество) оказалась модель случайного леса.

## Проверьте модель на тестовой выборке

**Логистическая регрессия**

In [ ]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.6842923794712286


**Случайный лес**

In [ ]:

model = RandomForestClassifier(max_depth=8, n_estimators=51, min_samples_leaf=2, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8040435458786936


**Дерево решений**

In [ ]:
model = DecisionTreeClassifier(max_depth=5, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7838258164852255


**Вывод:**

На тестовой выборке лучше всего показала себя модель Случайного леса.

## (бонус) Проверьте модели на адекватность

In [ ]:
dmb = DummyClassifier(strategy="stratified" ,random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество случайного прогноза (стратегия stratified):", acc)
print("Качество нашего прогноза:", accuracy)

Качество случайного прогноза (стратегия stratified): 0.536547433903577
Качество нашего прогноза: 0.7838258164852255


In [ ]:
dmb = DummyClassifier(strategy='most_frequent', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество случайного прогноза (стратегия most_frequent):", acc)
print("Качество нашего прогноза:", accuracy)

Качество случайного прогноза (стратегия most_frequent): 0.6842923794712286
Качество нашего прогноза: 0.7838258164852255


In [ ]:
dmb = DummyClassifier(strategy='uniform', random_state=12345)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество случайного прогноза (стратегия uniform):", acc)
print("Качество нашего прогноза:", accuracy)

Качество случайного прогноза (стратегия uniform): 0.4821150855365474
Качество нашего прогноза: 0.7838258164852255


## Общий вывод:

Были проверены три модели: логистическая регрессия, случайный лес, деревья решений. Наилучшее качество (0.804) достигается при использовании randomforest с гиперпараметрами max_depth=8 , _estimators=51 , min_samples_leaf=2